# Detailed Report

## Structure

In [ ]:
from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

from aiida.orm import load_node
import urlparse

import ipywidgets as ipw
from IPython.display import display, clear_output

import nglview

In [ ]:
url = urlparse.urlsplit(jupyter_notebook_url)
uuid = urlparse.parse_qs(url.query)['uuid'][0]
workcalc = load_node(uuid=str(uuid))

In [ ]:
viewer = nglview.NGLWidget()
clear_output()

In [ ]:
def refresh_structure_view():
    global viewer, structure
    if hasattr(viewer, "component_0"):
        #viewer.clear_representations()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_unitcell()
        cid = viewer.component_0.id
        viewer.remove_component(cid)
    viewer.add_component(nglview.ASEStructure(structure.get_ase())) # adds ball+stick
    viewer.add_unitcell()
    viewer.center_view()

In [ ]:
structure = workcalc.inp.structure
refresh_structure_view()

In [ ]:
display(ipw.VBox([viewer]))

## Isotherm plot

In [ ]:
def plot_bokeh(x, y, ids, title=None, xlabel=None, ylabel=None, clr_label=None, with_links=False):
    import bokeh.plotting as bpl
    import bokeh.models as bmd
    from bokeh.palettes import Viridis256
    import matplotlib as mpl
    from bokeh.io import output_notebook, show
    from bokeh.models import HoverTool
    output_notebook(hide_banner=True)
    
    rest_url = './show.ipynb'

    #cmap = bmd.LinearColorMapper(palette=Viridis256, low=min(colors), high=max(colors))
    #cbar = bmd.ColorBar(color_mapper=cmap, title=clr_label, location=(0, 0))
    
    hover = HoverTool(tooltips=[
        ("calc id", "@id"),
    ])
    
    fig = bpl.figure(
        toolbar_location=None,
        title=title,
        x_axis_label=xlabel,
        y_axis_label=ylabel,
        tools=['tap', 'zoom_in', 'zoom_out', 'pan', hover], 
        output_backend='webgl',
    )
    source = bmd.ColumnDataSource(data=dict(x=x, y=y, id=ids))
    
    fig.line('x', 'y', source=source)
    fig.circle('x', 'y', size=10, fill_color="white", source=source)
    #fig.scatter(x, y, radius=2.0, fill_color=colors, fill_alpha=0.6, line_color=None)
    #fig.add_layout(cbar, 'right')
        
    show(fig)

In [ ]:
isotherm = sorted(
    [
        (e['pressure'], e['loading'], e['calc_id']) 
        for e in workcalc.get_extra('isotherm')
    ],
    key=lambda x: x[0])

In [ ]:
p_units = workcalc.get_extra("pressure_units")
load_units = workcalc.get_extra("loading_units")

In [ ]:
p, load, calc_ids = zip(*isotherm)

In [ ]:
plot_bokeh(p, load, calc_ids,
           xlabel="Pressure [{}]".format(p_units),
           ylabel="Metane loading [{}]".format(load_units)
          )

## Computed properties

In [ ]:
def show_properties(workcalc):
    prop_list = [
        'density',
        'pore_accesible_volume',
        'unitcell_volume',
        'largest_free_sphere',
        'largest_included_free_sphere',
        'largest_included_sphere',
        'accessible_surface_area',
        'not_accessible_surface_area',
        'channel_surface_area',
        'deliverable_capacity',
        'henry_coefficient',
    ]

    # html table header
    html  = '<style>#aiida_results td,th {padding: 2px; min-width: 200px}</style>' 
    html += '<table border=1 id="aiida_results" style="margin:10px;"><tr>'
    html += '<th>Property</th>'
    html += '<th>Value</th>'
    html += '<th>Units</th>'
    html += '</tr>'

    for prop in prop_list:
        html += '<tr>'
        html += '<td>%s</td>' % prop.replace('_', ' ').title()
        html += '<td>%s</td>' % workcalc.get_extra(prop)
        html += '<td>%s</td>' % workcalc.get_extra(prop+'_units')
        html += '</td>'
        html += '</tr>'

    html += '</table>'

    results.value = html

In [ ]:
results = ipw.HTML()
show_properties(workcalc)
display(results)